## LibSBML Correction of Annotations

In [1]:
import sys
import os
import libsbml
import pandas as pd
import cobra

def get_model_path(extensions = ".fo.ch.mp.mcb.lt.re.ar.gpr.pw.gf1.gfmm.gf2.gfco3.circ.mcb2.sbo.eco.mp2.re"):
    return f"2.2/finegoldia_magna_ATCC_29328_2.2{extensions}.xml"

extensions = [".fo",".ch", ".mp", ".mcb", ".lt", ".re", ".ar", ".gpr", ".pw", ".gf1", ".gfmm", ".gf2", ".gfco3", ".circ", ".mcb2", ".sbo", ".eco", ".mp2", ".re", ""]

reader = libsbml.SBMLReader()
writer = libsbml.SBMLWriter()

doc = reader.readSBML(get_model_path())
model = doc.getModel()

model_cobra = cobra.io.read_sbml_model(get_model_path())

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [2]:
pathway_dict = dict()
for r in model_cobra.reactions:
    if "kegg.pathway" in r.annotation:
        pws =  r.annotation["kegg.pathway"]
        if not isinstance(pws, list):
            pws = [pws]
        pathway_dict[r.id] = pws
        r.annotation.pop("kegg.pathway")

In [3]:
def make_cv_term(link: str, qual_type):
    c = libsbml.CVTerm()
    c.setQualifierType(libsbml.BIOLOGICAL_QUALIFIER)
    c.setBiologicalQualifierType(qual_type)
    c.addResource(link)
    return c

def change_qual_annotation_reaction(model, lnk, qual_type, s_id):
    cv_term = make_cv_term(lnk, qual_type)
    
    for k in range(model.getReaction(s_id).getNumCVTerms()):
        rl = model.getReaction(s_id).getCVTerm(k).getResources().getLength()
        for j in range(rl):
            if model.getReaction(s_id).getCVTerm(k).getResources().getValue(j) == lnk:
                model.getReaction(s_id).getCVTerm(k).getResources().remove(j)
        
    model.getReaction(s_id).addCVTerm(cv_term)
        
    return model

In [5]:
from tqdm import tqdm

model.populateAllElementIdList()
for c_id, pws in tqdm(pathway_dict.items()):
    if not model.getAllElementIdList().contains(f"R_{c_id}"):
        print(c_id)
    for pw in pws:
        lnk = f"https://identifiers.org/kegg.pathway/{pw}"
        model = change_qual_annotation_reaction(model, lnk, libsbml.BQB_OCCURS_IN, f"R_{c_id}")

100%|██████████| 719/719 [00:01<00:00, 376.31it/s]


In [11]:
# Export model
doc.setModel(model)
writer.writeSBML(doc, f"{get_model_path()[:-4]}.acor.xml")

True